In [1]:
# !yes|pip uninstall spacy
# !pip install spacy==3.1.1
# !python -m spacy info
import spacy
print(spacy.__version__)  # 3.1.1


3.5.2


In [2]:
#!yes | pip uninstall spacy

!python -m spacy info
import spacy
print(spacy.__version__)  # 3.1.1

2023-06-08 12:57:38.910386: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.5.2                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.107+-x86_64-with-glibc2.31
Python version   3.10.11                       
Pipelines        en_core_web_sm (3.5.0)        

3.5.2


In [3]:
import spacy
import numpy as np
!yes|pip uninstall spacy-transformers
!pip install spacy-transformers

LANG = "ITA" # "ENG" or "ITA"

if LANG == "ITA":
  !python -m spacy download it_core_news_md
  nlp = spacy.load("it_core_news_md")
  import it_core_news_md
  pass
elif LANG == "ENG":
  !python -m spacy download en_core_web_lg --force
  nlp = spacy.load("en_core_web_lg")
  import en_core_web_lg
  nlp = en_core_web_lg.load()
  pass




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.2 MB/s eta 0:00:00
2023-06-08 12:58:10.478687: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 15.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_md')


In [4]:
import spacy
import spacy_transformers
import numpy as np

doc = nlp("Frase italiana.")

print([(w.text, w.pos_) for w in doc])
print(len(nlp.vocab))

[('Frase', 'NOUN'), ('italiana', 'ADJ'), ('.', 'PUNCT')]
383


In [10]:
import random

word1 = nlp('nero')
word2 = nlp('bianco')

similarity = word1.similarity(word2)
print(f'{word1} ~= {word2} {similarity}') # 0.835

print(len(nlp.vocab.strings))
list(sorted(random.choices(list(nlp.vocab.strings), k=10)))

nero ~= bianco 0.8355581480087845
681944


['357/1997',
 'Foppe',
 'Mathematics',
 'Moraldo',
 'RomaRarità',
 'Tree',
 'acra',
 'korugar',
 'migliorandolo',
 'tissi']

In [11]:
from numba import jit

@jit(nopython=True)
def cosine_similarity_numba(u:np.ndarray, v:np.ndarray):
    assert(u.shape[0] == v.shape[0])
    uv = 0
    uu = 0
    vv = 0
    for i in range(u.shape[0]):
        uv += u[i]*v[i]
        uu += u[i]*u[i]
        vv += v[i]*v[i]
    cos_theta = 1
    if uu != 0 and vv != 0:
        cos_theta = uv/np.sqrt(uu*vv)
    return cos_theta

In [47]:
def most_similar(word, topn=5, reverse=True):
  word = nlp.vocab[str(word)]
  queries = [
      w for w in nlp.vocab 
      if np.count_nonzero(w.vector)
  ]

  by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=reverse)
  return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

similar = most_similar("salume", topn=3)
print(similar)


dir(nlp.vocab["king"])
nlp.vocab["king"].has_vector
nlp

[('caserecci', 1.0), ('norcini', 1.0), ('salumieri', 1.0), ('casarecci', 1.0)]


In [48]:
def most_similar2(word, topn=5, reverse=True):
  word = nlp.vocab[str(word)]
  queries = [
      w for w in nlp.vocab 
      if np.count_nonzero(w.vector)
  ]

  by_similarity = sorted(queries, key=lambda w: cosine_similarity_numba(word.vector, w.vector), reverse=reverse)
  return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

similar = most_similar2("salume", topn=3)
print(similar)


dir(nlp.vocab["king"])
nlp.vocab["king"].has_vector
nlp

[('caserecci', 1.0), ('norcini', 1.0), ('salumieri', 1.0), ('casarecci', 1.0)]


In [15]:
# constansts

START_WORD_LENGTHS = [4,5,6]

MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 9

NUM_STEPS = 8
MIN_STEPS = 5



In [16]:
'''
Created on Feb 7, 2013

@author: olegs
'''

ROMAN_CONSTANTS = (
            ( "", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX" ),
            ( "", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC" ),
            ( "", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM" ),
            ( "", "M", "MM", "MMM", "",   "",  "-",  "",    "",     ""   ),
        )

ROMAN_SYMBOL_MAP = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)

CUTOFF = 4000
BIG_DEC = 2900
BIG_ROMAN = "MMCM"
ROMAN_NOUGHT = "nulla"

def digits(num):
    if num < 0:
        raise Exception('range error: negative numbers not supported')
    if num % 1 != 0.0:
        raise Exception('floating point numbers not supported')
    res = []
    while num > 0:
        res.append(num % 10)
        num //= 10
    return res

def toString(num, emptyZero=False):
    if num < CUTOFF:
        digitlist = digits(num)
        if digitlist:
            res = reversed([ ROMAN_CONSTANTS[order][digit] for order, digit in enumerate(digitlist) ])
            return "".join(res)
        else:
            return "" if emptyZero else ROMAN_NOUGHT 
    else:
        if num % 1 != 0.0:
            raise Exception('floating point numbers not supported')
        # For numbers over or equal the CUTOFF, the remainder of division by 2900
        # is represented as above, prepended with the multiples of MMCM (2900 in Roman),
        # which guarantees no more than 3 repetitive Ms.
        return BIG_ROMAN * (num // BIG_DEC) + toString(num % BIG_DEC, emptyZero=True)

def parse(numeral):
    numeral = numeral.upper()
    result = 0

    lastVal = 0
    lastCount = 0
    subtraction = False
    for symbol in numeral[::-1]:
        value = ROMAN_SYMBOL_MAP.get(symbol)
        if not value:
            return False
        if lastVal == 0:
            lastCount = 1
            lastVal = value
        elif lastVal == value:
            lastCount += 1
            # exceptions
        else:
            result += (-1 if subtraction else 1) * lastVal * lastCount
            subtraction = lastVal > value
            lastCount = 1
            lastVal = value
    
    result = result + (-1 if subtraction else 1) * lastVal * lastCount
    return True

In [17]:
import requests
import time

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'
dictionaryFileName = 'assets/Ita.txt'

    
def normalizeWord(word):
    return word.strip().upper()
    
LOAD_FROM_URL = True

t0 = time.time()

lines = []
if LOAD_FROM_URL:
    baseUrl = 'https://azrafe7.github.io/'
    f = requests.get(baseUrl + dictionaryFileName)
    lines = f.text.splitlines()
else:
    with open(dictionaryFileName) as f:
        lines = f.read().splitlines()

filteredWords = []
for line in lines:
    word = normalizeWord(line)
    length = len(word)
    if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH and not parse(word):
        filteredWords.append(word)

print("Dictionary loaded in {0:.2f}s".format(time.time() - t0))

print('{0}/{1}'.format(len(filteredWords), len(lines)))

print(filteredWords[:10])

Dictionary loaded in 0.87s
401123/710993
['AALENIANA', 'AALENIANE', 'AALENIANI', 'AALENIANO', 'ABABUA', 'ABACA', 'ABACETI', 'ABACETO', 'ABACHI', 'ABACHISTA']


In [19]:
for w in filteredWords:
  nlp.vocab[w.lower()]

In [55]:
import time

word = 'perche'

print(nlp.vocab[word].similarity(nlp.vocab['lupo']))

# t0 = time.time()
# similars = most_similar(word)
# print('{0:.2f}s'.format(time.time() - t0))
# display(similars)

t0 = time.time()
similars2 = most_similar2(word, reverse=True)
print('{0:.2f}s'.format(time.time() - t0))
display(similars2)


0.08700957894325256
2.23s


[('percio', 1.0),
 ('ergo', 0.8334516882896423),
 ('perché', 0.7591810822486877),
 ('cioe', 0.7424476146697998),
 ('pero', 0.7424476146697998)]

In [36]:
'-ci' in nlp.vocab
#[x.text for x in nlp.vocab]

False

In [ ]:
w = nlp.vocab['chaos']
sorted([x.text for x in nlp.vocab])
len([x.text for x in nlp.vocab])


In [ ]:
import spacy

from numpy import dot
from numpy.linalg import norm

parser = spacy.load("en_core_web_md")


In [ ]:
word = u"dog"
nasa = parser.vocab[word]

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

# gather all known words, take only the lowercased versions
allWords = list({w for w in parser.vocab if w.has_vector and w.orth_.islower() and w.is_alpha and w.lower_ != word})

# sort by similarity to NASA
allWords.sort(key=lambda w: cosine(w.vector, nasa.vector))
allWords.reverse()
print(f"Top 10 most similar words to {word}:")
for word in allWords[:20]:
    print(word.orth_, cosine(nasa.vector, word.vector))

[w.text for w in allWords]

In [ ]:
import random

def replace_synonyms(text):
    doc = nlp(text)
    new_doc = []
    for token in doc:
        print(token.has_vector, token.pos_)
        if token.has_vector and token.pos_ in ["NOUN", "VERB", "ADJ", "ADV", "PROPN"]:
            synonyms = [t for t in nlp.vocab if t.has_vector and t.similarity(token) > 0.2 and t.text != token.text]
            if synonyms:
                print(token.text, [s.text for s in synonyms])
                new_token = random.choice(synonyms)
                new_doc.append(new_token.text)
            else:
                new_doc.append(token.text)
        else:
            new_doc.append(token.text)
    return " ".join(new_doc)

text = "king key chain coffee water blind door nasa"
synonym_replaced_text = replace_synonyms(text)
print(synonym_replaced_text)
